In [1]:
from this_settings import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## 1. Load data

In [2]:
## CRSP
data_crsp = pd.read_csv(directory_rawdata+'data_crsp.csv')
data_crsp['date'] = pd.to_datetime(data_crsp['DATE'],format='%Y%m%d') +\
                    pd.offsets.MonthEnd(0)
data_crsp = data_crsp[['date','vwretd','vwretx','totval']].copy()

In [3]:
## Kenneth French's data
data_ff = pd.read_excel(directory_rawdata+'data_ff.xlsx')
data_ff.rename(columns={'Unnamed: 0':'ym'},
               inplace=True)
data_ff['ym'] = data_ff['ym'].astype(str)
data_ff['y'] = data_ff['ym'].str[:4]
data_ff['m'] = data_ff['ym'].str[-2:]
data_ff['date'] = pd.to_datetime(data_ff['y'] + '/' + data_ff['m'] + '/1') +\
                  pd.offsets.MonthEnd(0)
data_ff.columns = [x.lower() for x in data_ff.columns]
data_ff.rename(columns={'mkt-rf':'mkt_rf'},
               inplace=True)
data_ff['mkt'] = data_ff['mkt_rf'].copy() + data_ff['rf'].copy()

## Sub
data_ff = data_ff[['date','mkt_rf','mkt','rf','hml','smb']]

## /100 to 5% is 0.05
data_ff.set_index(['date'],
                  inplace=True)
data_ff = data_ff.copy()/100.0
data_ff.reset_index(inplace=True)

## 2. Merge and construct dividends monthly

In [4]:
## Merge
data_m = pd.merge(left=data_crsp,
                  right=data_ff,
                  on=['date'],
                  how='left')

In [5]:
## Construct dividends (monthly)
data_m.sort_values(['date'],
                 inplace=True)
data_m['totval_tm1'] = data_m['totval'].shift()
data_m['div'] = (data_m['vwretd'].copy() - data_m['vwretx'].copy()) *\
                 data_m['totval_tm1'].copy()

In [6]:
## Construct dividends, reinvested at market and cash
data_m['year'] = data_m['date'].dt.year
data_m.sort_values(['date'],
                   ascending=False,
                   inplace=True)

## First reinvested at market
data_m['vwretd_tp1'] = data_m.groupby(['year'])\
                             ['vwretd']\
                             .shift()
data_m['vwretd_tp1'] = data_m['vwretd_tp1'].copy() + 1.0
data_m['vwretd_tp1'].fillna(1.0,
                            inplace=True)
data_m['vwretd_cumprod'] = data_m.groupby(['year'])\
                                 ['vwretd_tp1']\
                                 .cumprod()
data_m['div_m'] = data_m['div'].copy() * data_m['vwretd_cumprod'].copy()

## Then reinvested at cash
data_m['rf_tp1'] = data_m.groupby(['year'])\
                         ['rf']\
                         .shift()
data_m['rf_tp1'] = data_m['rf_tp1'].copy() + 1.0
data_m['rf_tp1'].fillna(1.0,
                        inplace=True)
data_m['rf_cumprod'] = data_m.groupby(['year'])\
                              ['rf_tp1']\
                              .cumprod()
data_m['div_c'] = data_m['div'].copy() * data_m['rf_cumprod'].copy()

## 3. Resample to annual

In [7]:
## Get levels of dividends by sum, following Chen 2009
## Then get annual dividend growth by YoY change
data_a_div = data_m.groupby(['year'])\
                   [['div','div_m','div_c']]\
                   .sum()
data_a_div_yoy = data_a_div.copy()/data_a_div.copy().shift() - 1.0

## Levels of prices by last
## Then get returns
data_m.sort_values(['year'],
                   inplace=True)
data_a_p = data_m.groupby(['year'])\
                 [['totval']]\
                 .last()
data_a_ret = data_a_p.copy() / data_a_p.copy().shift() -1.0                 

In [8]:
## Get geometric annual return and dividend growth
## First get MoM growth in dividend
for this_col in ['div','div_c','div_m']:
    data_m[this_col+'_mom'] = data_m[this_col].copy() / data_m[this_col].copy().shift() - 1.0
    
## Then get annualized geometric
data_m_sub = data_m[['year','vwretd','div_mom','div_c_mom','div_m_mom']].copy()
data_m_sub.set_index(['year'],
                     inplace=True)
data_m_sub = data_m_sub.copy() + 1.0
data_m_sub.reset_index(inplace=True)
#data_a_geo = data_m_sub.resample('Y').prod()
data_a_geo = data_m_sub.groupby(['year'])\
                       [['vwretd','div_mom','div_c_mom','div_m_mom']]\
                       .prod()
#for this_col in data_a_geo.columns:
#    data_a_geo[this_col] = data_a_geo[this_col].apply(lambda x: np.power(x,1/12))
data_a_geo = data_a_geo.copy() - 1.0

## First year only has 9 months of data, so exclude
data_a_geo = data_a_geo.iloc[1:].copy()

In [9]:
(data_a_geo*100.0).agg(['mean','std']).round(2)

,vwretd,div_mom,div_c_mom,div_m_mom
mean,12.37,27.76,27.87,28.78
std,17.02,78.40,79.05,85.05


In [10]:
(data_a_div_yoy*100.0).agg(['mean','std']).round(2)

,div,div_m,div_c
mean,8.39,8.87,8.41
std,7.39,13.45,7.67


In [11]:
(data_a_ret*100.0).agg(['mean','std']).round(2)

,totval
mean,11.07
std,16.93


In [12]:
## Final annual data set
## Returns are geometric
## Dividend growth are as in Chen... so YoY, where each year is reinvested
## Dividend level are by last as in Chen
## Prices are by last
## Price dividend in like that
data_a = pd.merge(left=data_a_geo[['vwretd']],
                  right=data_a_div,
                  left_index=True,
                  right_index=True,
                  how='outer')
data_a = pd.merge(left=data_a,
                  right=data_a_div_yoy.add_suffix('_yoy'),
                  left_index=True,
                  right_index=True,
                  how='outer')
data_a = pd.merge(left=data_a,
                  right=data_a_p,
                  left_index=True,
                  right_index=True,
                  how='outer')
                  
## Drop 1945 due to way we annualized
data_a.dropna(inplace=True)

In [13]:
data_a.head()

,vwretd,div,div_m,div_c,div_yoy,div_m_yoy,div_c_yoy,totval
year,,,,,,,,
1946,-0.064212,2.599130e+06,2.452587e+06,2.602721e+06,0.187221,-0.027530,0.187346,68731812.2
1947,0.032932,3.231866e+06,3.362856e+06,3.239573e+06,0.243441,0.371146,0.244687,59903329.3
1948,0.021313,3.757384e+06,3.743055e+06,3.767355e+06,0.162605,0.113058,0.162917,61917670.8
1949,0.201093,4.139718e+06,4.708998e+06,4.157121e+06,0.101756,0.258063,0.103459,55347129.8
1950,0.304734,5.265458e+06,5.954802e+06,5.289761e+06,0.271936,0.264558,0.272458,68546982.0


In [14]:
## Create variables
data_a['log_ret'] = np.log(data_a['vwretd'].copy()+1.0)
data_a['log_pd'] = np.log(data_a['totval'].copy() / data_a['div'].copy())
data_a['log_pd_m'] = np.log(data_a['totval'].copy() / data_a['div_m'].copy())
data_a['log_pd_c'] = np.log(data_a['totval'].copy() / data_a['div_c'].copy())
data_a['log_div_yoy'] = np.log(data_a['div_yoy'].copy() + 1.0)
data_a['log_div_m_yoy'] = np.log(data_a['div_m_yoy'].copy() + 1.0)
data_a['log_div_c_yoy'] = np.log(data_a['div_c_yoy'].copy() + 1.0)

In [15]:
data_a.reset_index(inplace=True)
data_a.sort_values('year',inplace=True)
for this_col in ['log_pd','log_pd_m','log_pd_c']:
    data_a[this_col+'_tm1'] = data_a[this_col].copy().shift()

In [17]:
data_a['pd'] = data_a['totval'].copy() / data_a['div']
data_a['pd_m'] = data_a['totval'].copy() / data_a['div_m']
data_a['pd_c'] = data_a['totval'].copy() / data_a['div_c']

## 4. Out

In [18]:
data_a.to_parquet(directory_cleandata+'data_a.parquet')